In [28]:
import os
import dspy

In [29]:
lm = dspy.LM(
    model="groq/openai/gpt-oss-20b",
    api_key=os.environ.get("GROQ_API_KEY"),
)

dspy.configure(lm=lm)

In [30]:
class GenerateQuery(dspy.Signature):
    context = dspy.InputField(desc="It may contain relevant fact(s)")
    question = dspy.InputField(desc="Fact users wants to search for")
    query = dspy.OutputField(desc="What should be searched next?")

In [31]:
class GenerateAnswer(dspy.Signature):
    context = dspy.InputField(desc="It may contain relevant fact(s)")
    question = dspy.InputField(desc="Fact users wants to search for")
    answer = dspy.OutputField(desc="Concise answer in plain text - NO MARKDOWN")

In [32]:
embedder = dspy.Embedder(model="ollama/nomic-embed-text-v2-moe:latest")

In [47]:
class MultiHopRag(dspy.Module):
    def __init__(self, max_hops=3):
        super().__init__()

        self.max_hops = max_hops
        self.generate_query = [dspy.ChainOfThought(
            GenerateQuery) for _ in range(max_hops)]
        self.retrieve = dspy.Embeddings.from_saved(
            path="./embeddings", embedder=embedder)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)

    def forward(self, question):
        context = []
        for hop in range(self.max_hops):
            query_response = self.generate_query[hop](
                context=context, question=question)
            query = query_response.query
            passages = self.retrieve.__call__(query).passages
            context = context + passages
        
        pred = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=pred.answer)

In [48]:
question = "What tools are available that can be used for the purpose of Remote connection to MACs"
multi_hop = MultiHopRag(max_hops=3)

output = multi_hop(question=question)

In [49]:
output.answer

'Synergy (or comparable cross‑platform sharing tools) and Real VNC/VNC Viewer.'

In [50]:
set(output.context).__len__()

5

In [53]:
# lm.inspect_history(1)